<h1>Import libraries<h1>

In [ ]:
# Internal Library to register model
!pip install fosforml
from fosforml import *

In [2]:
pip install pyspark

     |████████████████████████████▍   | 280.9MB 97.4MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 204kB 80.6MB/s eta 0:00:01
Failed to build pyspark
    Running setup.py install for pyspark ... done
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
#estblishing connection between notebook and snowflake
import numpy as np
import pandas as pd
from tqdm import tqdm

import time
from snowflake.snowpark.session import Session
import configparser

import warnings
warnings.filterwarnings("ignore")

config = configparser.ConfigParser()
config.read("snowflake_connection.ini")

connection_parameters = {
    "user": f'{config["Snowflake"]["user"]}',
    "password": f'{config["Snowflake"]["password"]}',
    "account": f'{config["Snowflake"]["account"]}',
    "WAREHOUSE": f'{config["Snowflake"]["WAREHOUSE"]}',
    "DATABASE": f'{config["Snowflake"]["DATABASE"]}',
    "SCHEMA": f'{config["Snowflake"]["SCHEMA"]}'
}

def snowflake_connector(conn):
    try:
        session = Session.builder.configs(conn).create()
        print("connection successful!")
    except:
        raise ValueError("error while connecting with db")
    return session

session = snowflake_connector(connection_parameters)

connection successful!


In [2]:
sales_bangalore_2022 = session.table("SOL_ASSORTMENT_PLANNING_COMB").to_pandas()

In [3]:
sales_bangalore_2022.head()

,TRANSACTION_DATE,MNTH_CODE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,DISTRIBUTOR_CODE,PRODUCT_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,CATEGORY,VARIANT,BRAND
0,2023-07-01,202203,460.91,3,0.0027,460.909091,DB0706,PRD0041,OL81278,Montclair,New Jersey,Dolphin,Str1,Soap,Beauty Soap,Charcoal
1,2023-07-01,202203,277.14,32,8.32E-04,285.714286,DB0706,PRD0069,OL206924,Breckenridge,Colorado,Dolphin,Str2,Perfume and Deodrants,Female Deodrant,Arctic blue
2,2023-07-01,202203,636.36,4,0.0012,636.363636,DB0706,PRD0078,OL81756,Tucson,Arizona,City Center,Str1,Lotion,Head Lotion,Saffron
3,2023-07-01,202203,138.57,16,4.00E-04,142.857143,DB0209,PRD0147,OL238948,San Pedro,California,Orange,Str5,Kids Care,Baby Cream,Mint
4,2023-07-01,202203,142.86,16,4.16E-04,142.857143,DB0110,PRD0069,OL81622,Orange,Connecticut,Silver,Str5,Perfume and Deodrants,Female Deodrant,Arctic blue


In [4]:
train_final = session.table("SOL_ASSORTMENT_PLANNING_TRAIN_DATA_MODEL").to_pandas()
train_final.head(1)

,S_NO,UNIQUE_ID,FREQUENCYM,SALES_VALUE_AVG,SALES_UNITS_AVG,SALES_INDICATOR,SCHEME_AMOUNT_PERPRODUCT
0,0,PRD0002-OL10330,0.6,0.004323138,0.00040404,1,0.000465


In [5]:
train_final.columns = train_final.columns.str.lower()

In [6]:
train_final.head(1)

,s_no,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct
0,0,PRD0002-OL10330,0.6,0.004323138,0.00040404,1,0.000465


In [7]:
master_data = sales_bangalore_2022.copy()
master_data.head(1)

,TRANSACTION_DATE,MNTH_CODE,SALES_VALUE,SALES_UNITS,SALES_VOLUME,SALES_PTR_VALUE,DISTRIBUTOR_CODE,PRODUCT_CODE,OUTLET_CODE,CITY,STATE,COUNTY,STREET,CATEGORY,VARIANT,BRAND
0,2023-07-01,202203,460.91,3,0.0027,460.909091,DB0706,PRD0041,OL81278,Montclair,New Jersey,Dolphin,Str1,Soap,Beauty Soap,Charcoal


In [8]:
master_data.columns = master_data.columns.str.lower()
master_data.head(1)

,transaction_date,mnth_code,sales_value,sales_units,sales_volume,sales_ptr_value,distributor_code,product_code,outlet_code,city,state,county,street,category,variant,brand
0,2023-07-01,202203,460.91,3,0.0027,460.909091,DB0706,PRD0041,OL81278,Montclair,New Jersey,Dolphin,Str1,Soap,Beauty Soap,Charcoal


In [9]:
df1=master_data.copy
master_data['unique_id']=master_data['product_code']+'-'+ master_data['outlet_code']

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_final[['frequencym','sales_value_avg','scheme_amount_perproduct']],train_final['sales_indicator'],test_size=0.3,random_state=42)

In [11]:
import pickle

with open('logreg.pkl', 'rb') as model_file:
   loaded_model = pickle.load(model_file)
probs = loaded_model.predict_proba(X_test)
probs

array([[0.66778068, 0.33221932],
       [0.8336751 , 0.1663249 ],
       [0.44592937, 0.55407063],
       ...,
       [0.66800724, 0.33199276],
       [0.83363294, 0.16636706],
       [0.6676696 , 0.3323304 ]])

In [12]:
cutoff = 0.3
y_pred = (probs[:,1]>=cutoff).astype(int)
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.83      0.60      0.70     30799
           1       0.57      0.82      0.68     20399

    accuracy                           0.69     51198
   macro avg       0.70      0.71      0.69     51198
weighted avg       0.73      0.69      0.69     51198



In [13]:
test_final = session.table("SOL_ASSORTMENT_PLANNING_TEST_SET_MODEL").to_pandas()
test_final.columns = test_final.columns.str.lower()
test_final.head(1)


,s_no,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct
0,0,PRD0002-OL10330,0.666667,0.004323138,0.00040404,1,0.00156


In [14]:
X_test_unseen = test_final[['frequencym','sales_value_avg','scheme_amount_perproduct']]
y_test_unseen = test_final['sales_indicator']
print(X_test_unseen.shape,y_test_unseen.shape)
probs = loaded_model.predict_proba(X_test_unseen)
test_final['probablity']=probs[:,1]
y_pred = (probs[:,1]>=0.4).astype(int)
test_final

(189082, 3) (189082,)


,s_no,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct,probablity
0,0,PRD0002-OL10330,0.666667,0.004323138,0.00040404,1,0.001560,0.807117
1,1,PRD0002-OL10346,0.166667,0.001429593,0.00013468,0,0.001560,0.298748
2,2,PRD0002-OL10347,0.166667,0.001766519,0.00016835,0,0.001559,0.298690
3,3,PRD0002-OL10358,1.000000,0.01577123,0.001481481,1,0.001560,0.950099
4,4,PRD0002-OL10386,0.833333,0.00841699,0.000791246,1,0.001560,0.899377
...,...,...,...,...,...,...,...,...
189077,189077,PRD0168-OL97586,0.166667,0.001948381,0.00030303,1,0.001560,0.298657
189078,189078,PRD0168-OL97590,0.166667,0,0,0,0.001560,0.298999
189079,189079,PRD0168-OL97602,0.000000,0.001581375,0.000242424,0,0.001560,0.165792
189080,189080,PRD0168-OL97615,0.333333,0.001319099,0.00020202,1,0.001560,0.477312


In [15]:
test_final['prediction']= test_final['probablity'].apply(lambda x: 1  if x>=0.3 else 0)
test_final

,s_no,unique_id,frequencym,sales_value_avg,sales_units_avg,sales_indicator,scheme_amount_perproduct,probablity,prediction
0,0,PRD0002-OL10330,0.666667,0.004323138,0.00040404,1,0.001560,0.807117,1
1,1,PRD0002-OL10346,0.166667,0.001429593,0.00013468,0,0.001560,0.298748,0
2,2,PRD0002-OL10347,0.166667,0.001766519,0.00016835,0,0.001559,0.298690,0
3,3,PRD0002-OL10358,1.000000,0.01577123,0.001481481,1,0.001560,0.950099,1
4,4,PRD0002-OL10386,0.833333,0.00841699,0.000791246,1,0.001560,0.899377,1
...,...,...,...,...,...,...,...,...,...
189077,189077,PRD0168-OL97586,0.166667,0.001948381,0.00030303,1,0.001560,0.298657,0
189078,189078,PRD0168-OL97590,0.166667,0,0,0,0.001560,0.298999,0
189079,189079,PRD0168-OL97602,0.000000,0.001581375,0.000242424,0,0.001560,0.165792,0
189080,189080,PRD0168-OL97615,0.333333,0.001319099,0.00020202,1,0.001560,0.477312,1


In [16]:
test_final_static = test_final.copy()
print(classification_report(test_final['sales_indicator'],test_final['prediction']))

              precision    recall  f1-score   support

           0       0.79      0.75      0.77    124905
           1       0.56      0.61      0.58     64177

    accuracy                           0.70    189082
   macro avg       0.67      0.68      0.68    189082
weighted avg       0.71      0.70      0.71    189082



In [17]:
test_final.reset_index(drop=True)
test_final['OOS'] = test_final['probablity'].apply(lambda x:1 if x>0.3 else 0)

In [18]:
soq_master = master_data.groupby('unique_id').agg({'sales_units':'sum','mnth_code':'nunique'})
soq_master['SOQ'] = soq_master['sales_units']/soq_master['mnth_code']
soq_master.reset_index()
final_reco = pd.merge(soq_master,test_final,on='unique_id',how='inner')

In [19]:
final_reco = final_reco[['unique_id','prediction','SOQ','OOS']]
final_reco.head()

,unique_id,prediction,SOQ,OOS
0,PRD0002-OL10330,1,2.714286,1
1,PRD0002-OL10346,0,1.500000,0
2,PRD0002-OL10347,0,1.250000,0
3,PRD0002-OL10358,1,7.166667,1
4,PRD0002-OL10386,1,4.090909,1


In [20]:
ms_reco = final_reco.copy()
ms_reco.columns = ['unique_id', 'ms_flag', 'soq', 'oos_flag']

In [21]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [22]:
ms_reco[['product', 'outlet']] = ms_reco['unique_id'].str.split('-', expand=True)
ms_reco.head()

,unique_id,ms_flag,soq,oos_flag,product,outlet
0,PRD0002-OL10330,1,2.714286,1,PRD0002,OL10330
1,PRD0002-OL10346,0,1.500000,0,PRD0002,OL10346
2,PRD0002-OL10347,0,1.250000,0,PRD0002,OL10347
3,PRD0002-OL10358,1,7.166667,1,PRD0002,OL10358
4,PRD0002-OL10386,1,4.090909,1,PRD0002,OL10386


In [23]:
ms_reco=session.createDataFrame(
        ms_reco.values.tolist(),
        schema=ms_reco.columns.tolist())

In [24]:
#Writing back to Snowflake
ms_reco.write.mode("overwrite").save_as_table("SOL_CPG_DB.SOL_SALES_SCHEMA.SOL_ASSORTMENT_PREDICTION_TABLE")

In [25]:
req = requests.Request()
req.json = {"payload":X_test_unseen.head(1).to_json()}
print({'payload': req.json})

{'payload': {'payload': '{"frequencym":{"0":0.666666667},"sales_value_avg":{"0":"0.004323138"},"scheme_amount_perproduct":{"0":0.001559528}}'}}


In [26]:
from fosforml import *

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data = pd.DataFrame(eval(payload))
    prediction = pd.DataFrame(model.predict(data))
    return prediction.to_json()

In [27]:
print(score(loaded_model, req))

{"0":{"0":1.0}}


In [28]:
from fosforml.constants import MLModelFlavours


In [29]:
tmp = register_model(loaded_model, 
               score, 
               name="Assortment_Planning_Prediction", 
               description="prediction of assortment across retailers by product",
               flavour=MLModelFlavours.sklearn,
               model_type="regression",
               y_true=y_test,
               y_pred=y_pred, 
               prob=probs, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True, 
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train.tolist(),
               y_test=y_test.tolist(),
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['No Failure',' or Failure'],
               kyd=True, kyd_score = True)

Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%
